In [1]:
import sys
import random
import math
import itertools
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.special import lambertw
import seaborn as sns
import pickle
import warnings

import pyximport
pyximport.install(setup_args={"include_dirs":np.get_include()},
                  reload_support=True)
# import implementations as f
# import clip as clip

(None, <pyximport._pyximport3.PyxImportMetaFinder at 0x1065718a0>)

In [3]:
# solve for gamma
L = 1
U = 500
beta = 20
epsilon = 3
gamma = clip.solve_gamma((1+epsilon), U, L, beta)

In [4]:
print(gamma)

(465.6095222449376+0j)


In [5]:
epsilon = 0.0001
gamma = clip.solve_gamma((1+epsilon), U, L, beta)
print(gamma)

(499.9988525004798+0j)


In [1]:
import metric

# test the metric
m = metric.MetricSpace()

# print(m.avg_distances)

In [3]:
import carbonTraces

names = m.get_names()

# get the diameter
D = m.diameter()

# get the distance matrix
simplex_names, c_simplex, simplex_distances = m.generate_simplex_distances()
dim = len(simplex_names)

# get the weight vector, the c vector, and the name vector
weights = m.get_weight_vector()
c_vector, name_vector = m.get_unit_c_vector()

# get the carbon trace
datetimes, carbon_vector = carbonTraces.get_numpy(m)

In [4]:
print(name_vector)

print(carbon_vector)

['root', "['us-west-1', 'il-central-1', 'us-east-1', 'af-south-1', 'eu-west-2', 'ap-south-2', 'eu-west-3', 'eu-north-1', 'ca-central-1', 'eu-central-1', 'sa-east-1', 'ap-northeast-2', 'us-west-2', 'ap-southeast-2']", "['us-west-1', 'il-central-1', 'us-east-1', 'af-south-1', 'eu-west-2', 'ap-south-2', 'eu-west-3', 'eu-north-1', 'ca-central-1', 'eu-central-1', 'sa-east-1', 'ap-northeast-2', 'us-west-2', 'ap-southeast-2']", 'ap-southeast-2', "['us-west-1', 'il-central-1', 'eu-west-2', 'af-south-1', 'ap-south-2', 'eu-west-3', 'eu-north-1', 'eu-central-1', 'sa-east-1', 'ap-northeast-2', 'us-west-2']", "['us-west-1', 'il-central-1', 'eu-west-2', 'af-south-1', 'ap-south-2', 'eu-west-3', 'eu-north-1', 'eu-central-1', 'sa-east-1', 'ap-northeast-2', 'us-west-2']", 'il-central-1', 'af-south-1', 'sa-east-1', 'ap-south-2', 'ca-central-1', 'us-east-1', 'ap-southeast-2 OFF', "['eu-north-1', 'eu-west-2', 'eu-west-3', 'eu-central-1', 'ap-northeast-2']", 'eu-north-1', 'eu-central-1', "['us-west-1', 'us-

In [3]:
m.print_tree()

 ['eu-central-1', 'ap-south-2', 'us-west-1', 'sa-east-1', 'il-central-1', 'us-east-1', 'af-south-1', 'us-west-2', 'eu-west-3', 'ca-central-1', 'eu-north-1', 'eu-west-2', 'ap-southeast-2', 'ap-northeast-2']


In [4]:
print(c_vector)

[0]


In [5]:
print(m.unit_c_vector)

[0]


In [5]:
# Load the carbon traces into a dict of pandas dataframe
dfs = {}
for name in names:
    df = pd.read_csv(f"carbon-data/{name}.csv", parse_dates=["datetime"])
    # keep only the columns we need
    df = df[["datetime", "carbon_intensity_avg"]]
    dfs[name] = df

In [4]:
print(carbon_vector)

[[0.]
 [0.]
 [0.]
 ...
 [0.]
 [0.]
 [0.]]


In [9]:
name_vector = m.name_vector.copy()
print(name_vector)
c_vector = m.unit_c_vector.copy()
print(c_vector)

for i, name in enumerate(name_vector):
    if "[" in name: 
        name_vector[i] = "none" 
    if "OFF" in name:
        name_vector[i] = "none"
    if name == 'root':
        name_vector[i] = "none" 

print(name_vector)

['root']
[0]
['none']


In [ ]:
# create a numpy matrix where each column is the corresponding carbon intensity trace for that region
X = np.zeros((len(dfs["us-east-1"]), len(name_vector)))

for i, name in enumerate(name_vector):
    if name == "none":
        continue
    print(dfs[name]["carbon_intensity_avg"].values)
    carbon = dfs[name]["carbon_intensity_avg"].values
    X[i, :] = np.array(carbon)

In [1]:
import metric
m = metric.MetricSpace()

In [2]:
m = metric.MetricSpace()

In [8]:
import random
import numpy as np

distances = np.array([1, 2, 3, 4])

times = 4

simplex_distance_rands = [distances] 
for _ in range(times-1):
    simplex_distance_rands.append(np.random.permutation(distances))
# shuffle the distance list
random.shuffle(simplex_distance_rands)

print(simplex_distance_rands)

[array([1, 2, 3, 4]), array([2, 1, 4, 3]), array([4, 3, 1, 2]), array([2, 1, 3, 4])]


In [9]:
# format these distances into a list of length exactly T
simplex_distance_list = []
repeat = 24 // times
for distances in simplex_distance_rands:
    simplex_distance_list.extend([distances] * repeat)
# add the remaining distances  
simplex_distance_list.extend([simplex_distance_list[-1]] * (24 - len(simplex_distance_list)))

In [10]:
print(simplex_distance_list)

[array([1, 2, 3, 4]), array([1, 2, 3, 4]), array([1, 2, 3, 4]), array([1, 2, 3, 4]), array([1, 2, 3, 4]), array([1, 2, 3, 4]), array([2, 1, 4, 3]), array([2, 1, 4, 3]), array([2, 1, 4, 3]), array([2, 1, 4, 3]), array([2, 1, 4, 3]), array([2, 1, 4, 3]), array([4, 3, 1, 2]), array([4, 3, 1, 2]), array([4, 3, 1, 2]), array([4, 3, 1, 2]), array([4, 3, 1, 2]), array([4, 3, 1, 2]), array([2, 1, 3, 4]), array([2, 1, 3, 4]), array([2, 1, 3, 4]), array([2, 1, 3, 4]), array([2, 1, 3, 4]), array([2, 1, 3, 4])]


In [11]:
print(len(simplex_distance_list))

24
